# Dashboards and Widgets


In [1]:
from pathlib import Path

files = Path("files")  # location of test files

In [2]:
from metador_core.plugins import schemas

BibMeta = schemas["core.bib"]
DBMeta = schemas["core.dashboard"]
ImgMeta = schemas["core.imagefile"]

In [3]:
from metador_core.plugins import harvesters
from metador_core.harvester import harvest, metadata_loader, file_harvester_pipeline

HrvFile = harvesters["core.file.generic"]
HrvImgDim = harvesters["core.imagefile.dim"]

ImgMetaLoader = metadata_loader(ImgMeta, use_sidecar=True)
image_pipeline = file_harvester_pipeline(HrvFile, HrvImgDim, ImgMetaLoader)

In [4]:
from metador_core.container import MetadorContainer
from metador_core.packer.utils import pack_file

filepath = Path("my-dashboardable-container.h5")
if filepath.is_file():
    filepath.unlink()  # otherwise might fail to overwrite if previous attempt not closed

with MetadorContainer(filepath, "w") as m:
    m.meta["core.bib"] = BibMeta.parse_file(files / "test.bibmeta.yaml")
    
    imgmeta = harvest(ImgMeta, image_pipeline(files / "test.png"))
    n = pack_file(m, files / "test.png", target="foo/bar", metadata=imgmeta)
    n.meta[DBMeta] = DBMeta.show(group=1)

    n = pack_file(m, files / "test.csv", target="csvfile")
    n.meta[DBMeta] = DBMeta.show(group=1)

    n = pack_file(m, files / "test.tsv")
    n.meta[DBMeta] = DBMeta()
    
    n = pack_file(m, files / "test.pdf")
    n.meta[DBMeta] = DBMeta.show(group=2, priority=10)
    
    n = pack_file(m, files / "test.json")
    n.meta[DBMeta] = DBMeta.show(group=2, priority=5)
    
    n = pack_file(m, files / "test.md")
    n.meta[DBMeta] = DBMeta()
    
    n = pack_file(m, files / "test.html")
    n.meta[DBMeta] = DBMeta.show(
        [DBMeta.widget(widget_name="core.file.text.code", group=2), DBMeta.widget(group=2)]
    )


In [5]:
from metador_core.widget.jupyter import Previewable
from metador_core.plugins import widgets

In [6]:
m = Previewable(MetadorContainer("my-dashboardable-container.h5"))

 * Serving Flask app 'metador_core.widget.jupyter.standalone'
 * Debug mode: off


In [7]:
widgets["core.file.image"](m["foo/bar"]).show()

PNG(str, alt_text='Sorry. Something w...)

In [8]:
widgets["core.file.text.code"](m["/test.html"]).show()

CodeEditor(disabled=True, filename='test.html', readonly=True, value='<html>\n    <...)

In [9]:
widgets["core.file.csv"](m["/csvfile"]).show()

Tabulator(disabled=True, value=           Event  ...)

In [10]:
widgets["core.file.pdf"](m["/test.pdf"]).show()

PDF(str)

In [11]:
from metador_core.widget.dashboard import Dashboard

Dashboard(m).show()

FlexBox(align_content='space-evenly', align_items='center', flex_direction='column', justify_content='space-evenly', objects=[FlexBox(align_content='sp...], sizing_mode='scale_both')

In [12]:
m.close()  # important! or we might get file lock issues